<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>


# Chat with your pdfs with ColBERT, langchain, and Vespa

This notebook illustrates using [Vespa streaming mode](https://docs.vespa.ai/en/streaming-search.html)
to build cost-efficient RAG applications over naturally sharded data. It also demonstrates how you can now use ColBERT ranking natively in Vespa, which can now handle the ColBERT embedding process for you with no custom code!

You can read more about Vespa vector streaming search in these blog posts:

- [Announcing vector streaming search: AI assistants at scale without breaking the bank](https://blog.vespa.ai/announcing-vector-streaming-search/)
- [Yahoo Mail turns to Vespa to do RAG at scale](https://blog.vespa.ai/yahoo-mail-turns-to-vespa-to-do-rag-at-scale/)
- [Hands-On RAG guide for personal data with Vespa and LLamaIndex](https://blog.vespa.ai/scaling-personal-ai-assistants-with-streaming-mode/)
- [Turbocharge RAG with LangChain and Vespa Streaming Mode for Sharded Data](https://blog.vespa.ai/turbocharge-rag-with-langchain-and-vespa-streaming-mode/)

This notebook is also available in blog form: [TODO FIXME](https://blog.vespa.ai/)

### TLDR; Vespa streaming mode for partitioned data

Vespa's streaming search solution enables you to integrate a user ID (or any sharding key) into the Vespa document ID.
This setup allows Vespa to efficiently group each user's data on a small set of nodes and the same disk chunk.
Streaming mode enables low latency searches on a user's data without keeping data in memory.

The key benefits of streaming mode:

- Eliminating compromises in precision introduced by approximate algorithms
- Achieve significantly higher write throughput, thanks to the absence of index builds required for supporting approximate search.
- Optimize efficiency by storing documents, including tensors and data, on disk, benefiting from the cost-effective economics of storage tiers.
- Storage cost is the primary cost driver of Vespa streaming mode; no data is in memory. Avoiding memory usage lowers deployment costs significantly.


### Connecting LangChain Retriever with Vespa for Context Retrieval from PDF Documents

In this notebook, we seamlessly integrate a custom [LangChain](https://python.langchain.com/docs/get_started/introduction)
[retriever](https://python.langchain.com/docs/modules/data_connection/) with a Vespa app,
leveraging Vespa's streaming mode to extract meaningful context from PDF documents.

The workflow

- Define and deploy a Vespa [application package](https://docs.vespa.ai/en/application-packages.html) using PyVespa.
- Utilize [LangChain PDF Loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf) to download and parse PDF files.
- Leverage [LangChain Document Transformers](https://python.langchain.com/docs/modules/data_connection/document_transformers/)
to convert each PDF page into multiple text chunks.
- Feed the transformer representation to the running Vespa instance
- Employ Vespa's built-in ColBERT embedder functionality (using an open-source embedding model) for embedding the text chunks, resulting in a multi-vector representation per chunk
- Develop a custom [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/) to enable seamless retrieval for any unstructured text query.


![Overview](https://blog.vespa.ai/assets/2023-12-08-turbocharge-rag-with-langchain-and-vespa-streaming-mode/turbocharge-RAG-vespa-streaming.png)

Let's get started! First, install dependencies:

In [1]:
!pip3 install -U pyvespa langchain langchain-openai pypdf openai

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-brew-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-brew packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


## Sample data
We love [ColBERT](https://blog.vespa.ai/pretrained-transformer-language-models-for-search-part-3/), so
we'll use a few COlBERT related papers as examples of PDFs in this notebook.

In [2]:
def sample_pdfs():
    return [
        {
            "title": "ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction",
            "url": "https://arxiv.org/pdf/2112.01488.pdf",
            "authors": "Keshav Santhanam, Omar Khattab, Jon Saad-Falcon, Christopher Potts, Matei Zaharia"
        },
        {
            "title": "ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT",
            "url": "https://arxiv.org/pdf/2004.12832.pdf",
            "authors": "Omar Khattab, Matei Zaharia"
        },
        {
            "title": "On Approximate Nearest Neighbour Selection for Multi-Stage Dense Retrieval",
            "url": "https://arxiv.org/pdf/2108.11480.pdf",
            "authors": "Craig Macdonald, Nicola Tonellotto"
        },
        {
            "title": "A Study on Token Pruning for ColBERT",
            "url": "https://arxiv.org/pdf/2112.06540.pdf",
            "authors": "Carlos Lassance, Maroua Maachou, Joohee Park, Stéphane Clinchant"
        },
        {
            "title": "Pseudo-Relevance Feedback for Multiple Representation Dense Retrieval",
            "url": "https://arxiv.org/pdf/2106.11251.pdf",
            "authors": "Xiao Wang, Craig Macdonald, Nicola Tonellotto, Iadh Ounis"
        }

    ]

## Definining the Vespa application
[PyVespa](https://pyvespa.readthedocs.io/en/latest/) helps us build the [Vespa application package](https://docs.vespa.ai/en/application-packages.html).
A Vespa application package consists of configuration files, schemas, models, and code (plugins).   

First, we define a [Vespa schema](https://docs.vespa.ai/en/schemas.html) with the fields we want to store and their type.

In [3]:
from vespa.package import Schema, Document, Field, FieldSet
pdf_schema = Schema(
            name="pdf",
            mode="streaming",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="title", type="string", indexing=["summary", "index"]),
                    Field(name="url", type="string", indexing=["summary", "index"]),
                    Field(name="authors", type="array<string>", indexing=["summary", "index"]),
                    Field(name="metadata", type="map<string,string>", indexing=["summary", "index"]),
                    Field(name="page", type="int", indexing=["summary", "attribute"]),
                    Field(name="chunks", type="array<string>", indexing=["summary", "index"]),

                    Field(name="embedding", type="tensor<bfloat16>(chunk{}, x[384])",
                        indexing=['input chunks', 'for_each { "passage: " . (input title || "") . " " . ( _ || "") }', "embed e5", "attribute"],
                        attribute=["distance-metric: angular"],
                        is_document_field=False
                    ),

                    Field(name="colbert", type="tensor<int8>(context{}, token{}, v[16])",
                        indexing=['input chunks', "embed colbert context", "attribute"],
                        is_document_field=False
                    )
                ],
            ),
            fieldsets=[
                FieldSet(name = "default", fields = ["title", "chunks"])
            ]
)

The above defines our `pdf` schema using mode `streaming`. Most fields are straightforward,  but take a note of:

- `metadata` using `map<string,string>` - here we can store and match over page level metadata extracted by the PDF parser.
- `chunks` using `array<string>`, these are the text chunks that we use langchain document transformers for.
- The `embedding` field of type `tensor<bfloat16>(chunk{},x[384])` allows us to store and search a single 384-dimensional embeddings per chunk in the same document
- The `colbert` field of type `tensor<int8>(dt{}, x[16])` stores the ColBERT embedding, retaining a (quantized) per-token representation of the text.

The observant reader might have noticed the `e5` and `colbert` arguments to the `embed` expression in the above `embedding` field.
The `e5` argument references a component of the type [hugging-face-embedder](https://docs.vespa.ai/en/embedding.html#huggingface-embedder), and `colbert` references the new [cobert-embedder](https://docs.vespa.ai/en/embedding.html#colbert-embedder). We configure
the application package and its name with the `pdf` schema and the `e5` and `colbert` embedder components.

In [4]:
from vespa.package import ApplicationPackage, Component, Parameter

vespa_app_name = "pdfs"
vespa_application_package = ApplicationPackage(
        name=vespa_app_name,
        schema=[pdf_schema],
        components=[
            Component(id="e5", type="hugging-face-embedder",
              parameters=[
                  Parameter("transformer-model", {"url": "https://huggingface.co/intfloat/e5-small-v2/resolve/main/model.onnx"}),
                  Parameter("tokenizer-model", {"url": "https://huggingface.co/intfloat/e5-small-v2/raw/main/tokenizer.json"})
              ]
            ),
            Component(id="colbert", type="colbert-embedder",
              parameters=[
                  Parameter("transformer-model", {"url": "https://huggingface.co/colbert-ir/colbertv2.0/resolve/main/model.onnx"}),
                  Parameter("tokenizer-model", {"url": "https://huggingface.co/colbert-ir/colbertv2.0/raw/main/tokenizer.json"})
              ]
            )
        ]
)

In the last step, we configure [ranking](https://docs.vespa.ai/en/ranking.html) by adding `rank-profile`'s to the schema.

Vespa supports [phased ranking](https://docs.vespa.ai/en/phased-ranking.html) and has a rich set of built-in [rank-features](https://docs.vespa.ai/en/reference/rank-features.html), including many
text-matching features such as:

- [BM25](https://docs.vespa.ai/en/reference/bm25.html).
- [nativeRank](https://docs.vespa.ai/en/reference/nativerank.html) and many more.

Users can also define custom functions using [ranking expressions](https://docs.vespa.ai/en/reference/ranking-expressions.html). The following defines a `colbert` Vespa ranking profile which uses the `e5` embedding in the first phase, and the `max_sim` function in the second phase. The `max_sim` function performs the _late interaction_ for the ColBERT ranking, and is by default applied to the top 100 documents from the first phase.

In [5]:
from vespa.package import RankProfile, Function, FirstPhaseRanking, SecondPhaseRanking

colbert = RankProfile(
    name="colbert",
    inputs=[
        ("query(q)", "tensor<float>(x[384])"),
        ("query(qt)", "tensor<float>(querytoken{}, v[128])")
        ],
    functions=[
        Function(
            name="cos_sim",
            expression="closeness(field, embedding)"
        ),
        Function(
            name="max_sim_per_context",
            expression="""
                sum(
                    reduce(
                        sum(
                            query(qt) * unpack_bits(attribute(colbert)) , v
                        ),
                        max, token
                    ),
                    querytoken
                )
            """
        ),
        Function(
            name="max_sim",
            expression="reduce(max_sim_per_context, max, context)"
        )
    ],
    first_phase=FirstPhaseRanking(
        expression="cos_sim"
    ),
    second_phase=SecondPhaseRanking(
        expression="max_sim"
    ),
    match_features=["cos_sim", "max_sim", "max_sim_per_context"]
)
pdf_schema.add_rank_profile(colbert)

For an example of a `hybrid` rank-profile which combines semantic search with traditional text retrieval such as BM25, see the previous blog post: [Turbocharge RAG with LangChain and Vespa Streaming Mode for Sharded Data](https://blog.vespa.ai/turbocharge-rag-with-langchain-and-vespa-streaming-mode/)

## Deploy the application to Vespa Cloud

With the configured application, we can deploy it to [Vespa Cloud](https://cloud.vespa.ai/en/).
It is also possible to deploy the app using docker; see the [Hybrid Search - Quickstart](https://pyvespa.readthedocs.io/en/latest/getting-started-pyvespa.html) guide for
an example of deploying it to a local docker container.

Install the Vespa CLI using [homebrew](https://brew.sh/) - or download a binary from GitHub as demonstrated below.

In [6]:
!brew install vespa-cli

To reinstall 8.309.34, run:
  brew reinstall vespa-cli


Alternatively, if running in Colab, download the Vespa CLI:

In [7]:
import os
import requests
res = requests.get(url="https://api.github.com/repos/vespa-engine/vespa/releases/latest").json()
os.environ["VERSION"] = res["tag_name"].replace("v", "")
!curl -fsSL https://github.com/vespa-engine/vespa/releases/download/v${VERSION}/vespa-cli_${VERSION}_linux_amd64.tar.gz | tar -zxf -
!ln -sf /content/vespa-cli_${VERSION}_linux_amd64/bin/vespa /bin/vespa

ln: /bin/vespa: Operation not permitted


To deploy the application to Vespa Cloud we need to create a tenant in the Vespa Cloud:

Create a tenant at [console.vespa-cloud.com](https://console.vespa-cloud.com/) (unless you already have one).
This step requires a Google or GitHub account, and will start your [free trial](https://cloud.vespa.ai/en/free-trial).
Make note of the tenant name, it is used in the next steps.

### Configure Vespa Cloud date-plane security

Create Vespa Cloud data-plane mTLS cert/key-pair. The mutual certificate pair is used to talk to your Vespa cloud endpoints. See [Vespa Cloud Security Guide](https://cloud.vespa.ai/en/security/guide) for details.

We save the paths to the credentials for later data-plane access without using pyvespa APIs.

In [8]:
import os

os.environ["TENANT_NAME"] = "vespa-team" # Replace with your tenant name

vespa_cli_command = f'vespa config set application {os.environ["TENANT_NAME"]}.{vespa_app_name}'

!vespa config set target cloud
!{vespa_cli_command}
!vespa auth cert -N

Error: private key '/Users/andreer/.vespa/vespa-team.pdfs.default/data-plane-private-key.pem' already exists
Hint: Use -f flag to force overwriting


Validate that we have the expected data-plane credential files:

In [9]:
from os.path import exists
from pathlib import Path

cert_path = Path.home() / ".vespa" / f"{os.environ['TENANT_NAME']}.{vespa_app_name}.default/data-plane-public-cert.pem"
key_path = Path.home() / ".vespa" / f"{os.environ['TENANT_NAME']}.{vespa_app_name}.default/data-plane-private-key.pem"

if not exists(cert_path) or not exists(key_path):
    print("ERROR: set the correct paths to security credentials. Correct paths above and rerun until you do not see this error")

Note that the subsequent Vespa Cloud deploy call below will add `data-plane-public-cert.pem` to the application before deploying it to Vespa Cloud, so that
you have access to both the private key and the public certificate. At the same time, Vespa Cloud only knows the public certificate.

### Configure Vespa Cloud control-plane security

Authenticate to generate a tenant level control plane API key for deploying the applications to Vespa Cloud, and save the path to it.

The generated tenant api key must be added in the Vespa Console before attemting to deploy the application.

```
To use this key in Vespa Cloud click 'Add custom key' at
https://console.vespa-cloud.com/tenant/TENANT_NAME/account/keys
and paste the entire public key including the BEGIN and END lines.
```

In [10]:
!vespa auth api-key

from pathlib import Path
api_key_path = Path.home() / ".vespa" / f"{os.environ['TENANT_NAME']}.api-key.pem"

Error: refusing to overwrite '/Users/andreer/.vespa/vespa-team.api-key.pem'
Hint: Use -f to overwrite it

This is your public key:
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEQZ1UVQJ2kRlGR98vpNruWwYTYRFH
8kpQ8QRLpN/4S5LrQ2BtzBy2ETcJ0ZP5CQ5q3pwTj17bgdVB7OzlW6Kb9Q==
-----END PUBLIC KEY-----

Its fingerprint is:
be:ee:89:98:15:a1:d4:44:d1:79:08:01:e8:bf:0e:f2

To use this key in Vespa Cloud click 'Add custom key' at
https://console.vespa-cloud.com/tenant/vespa-team/account/keys
and paste the entire public key including the BEGIN and END lines.


### Deploy to Vespa Cloud

Now that we have data-plane and control-plane credentials ready, we can deploy our application to Vespa Cloud!

`PyVespa` supports deploying apps to the [development zone](https://cloud.vespa.ai/en/reference/environments#dev-and-perf).

>Note: Deployments to dev and perf expire after 7 days of inactivity, i.e., 7 days after running deploy. This applies to all plans, not only the Free Trial. Use the Vespa Console to extend the expiry period, or redeploy the application to add 7 more days.

In [11]:
from vespa.deployment import VespaCloud

def read_secret():
    """Read the API key from the environment variable. This is
    only used for CI/CD purposes."""
    t = os.getenv("VESPA_TEAM_API_KEY")
    if t:
        return t.replace(r"\n", "\n")
    else:
        return t

vespa_cloud = VespaCloud(
    tenant=os.environ["TENANT_NAME"],
    application=vespa_app_name,
    key_content=read_secret() if read_secret() else None,
    key_location=api_key_path,
    application_package=vespa_application_package)

Now deploy the app to Vespa Cloud dev zone.

The first deployment typically takes 2 minutes until the endpoint is up.

In [12]:
from vespa.application import Vespa
app:Vespa = vespa_cloud.deploy()

Deployment started in run 8 of dev-aws-us-east-1c for vespa-team.pdfs. This may take a few minutes the first time.
INFO    [11:42:14]  Deploying platform version 8.314.57 and application dev build 8 for dev-aws-us-east-1c of default ...
INFO    [11:42:14]  Using CA signed certificate version 1
INFO    [11:42:14]  Using 1 nodes in container cluster 'pdfs_container'
INFO    [11:42:19]  Session 285245 for tenant 'vespa-team' prepared and activated.
INFO    [11:42:23]  ######## Details for all nodes ########
INFO    [11:42:31]  h88971b.dev.aws-us-east-1c.vespa-external.aws.oath.cloud: expected to be UP
INFO    [11:42:31]  --- platform vespa/cloud-tenant-rhel8:8.314.57 <-- :
INFO    [11:42:31]  --- storagenode on port 19102 has not started 
INFO    [11:42:31]  --- searchnode on port 19107 has not started 
INFO    [11:42:31]  --- distributor on port 19111 has not started 
INFO    [11:42:31]  --- metricsproxy-container on port 19092 has not started 
INFO    [11:42:31]  h88974c.dev.aws-us-east

### Processing PDFs with LangChain

[LangChain](https://python.langchain.com/) has a rich set of [document loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/) that can be used to load and process various file formats. In this notebook, we use the [PyPDFLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pypdf).

We also want to split the extracted text into *chunks* using a [text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/). Most text embedding models have limited input lengths (typically less than 512 language model tokens, so splitting the text
into multiple chunks that fits into the context limit of the embedding model is a common strategy.

For embedding text data, models based on the Transformer architecture have become the de facto standard. A challenge with Transformer-based models is their input length limitation due to the quadratic self-attention computational complexity. For example, a popular open-source text embedding model like
[e5](https://huggingface.co/intfloat/e5-small) has an absolute maximum input length of 512 wordpiece tokens. In addition to
the technical limitation, trying to fit more tokens than used during fine-tuning of the model will impact the quality of the vector representation.

One can view this text embedding encoding as a lossy compression technique, where variable-length texts are compressed
into a fixed dimensional vector representation.

Although this compressed representation is very useful, it can be imprecise especially as the size of the text increases. By adding the ColBERT embedding, we also retain token-level information which retains more of the original meaning of the text and allows the richer _late interaction_ between the query and the document text.

In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024, #chars, not llm tokens
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

The following iterates over the `sample_pdfs` and performs the following:

- Load the URL and extract the text into pages. A page is the retrievable unit we will use in Vespa
- For each page, use the text splitter to split the text into chunks. The chunks are represented as an `array<string>` in the Vespa schema
- Create the page level Vespa `fields`, note that we duplicate some content like the title and URL into the page level representation.

In [14]:
import hashlib
import unicodedata
def remove_control_characters(s):
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

my_docs_to_feed = []
for pdf in sample_pdfs():
    url = pdf['url']
    loader = PyPDFLoader(url)
    pages = loader.load_and_split()
    for index, page in enumerate(pages):
        source = page.metadata['source']
        chunks = text_splitter.transform_documents([page])
        text_chunks = [chunk.page_content for chunk in chunks]
        text_chunks = [remove_control_characters(chunk) for chunk in text_chunks]
        page_number = index + 1
        vespa_id = f"{url}#{page_number}"
        hash_value = hashlib.sha1(vespa_id.encode()).hexdigest()
        fields = {
            "title" : pdf['title'],
            "url" : url,
            "page": page_number,
            "id": hash_value,
            "authors": [a.strip() for a in pdf['authors'].split(",")],
            "chunks": text_chunks,
            "metadata": page.metadata
        }
        my_docs_to_feed.append(fields)


Now that we have parsed the input PDFs and created a list of pages that we want to add to Vespa, we must format the
list into the format that PyVespa accepts. Notice the `fields`, `id` and `groupname` keys. The `groupname` is the
key that is used to shard and co-locate the data and is only relevant when using Vespa with streaming mode.

In [15]:
from typing import Iterable
def vespa_feed(user:str) -> Iterable[dict]:
    for doc in my_docs_to_feed:
        yield {
            "fields": doc,
            "id": doc["id"],
            "groupname": user
        }

In [16]:
my_docs_to_feed[0]

{'title': 'ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction',
 'url': 'https://arxiv.org/pdf/2112.01488.pdf',
 'page': 1,
 'id': 'a731a839198de04fa3d1a3cee6890d0d170ab025',
 'authors': ['Keshav Santhanam',
  'Omar Khattab',
  'Jon Saad-Falcon',
  'Christopher Potts',
  'Matei Zaharia'],
 'chunks': ['ColBERTv2:Effective and Efﬁcient Retrieval via Lightweight Late InteractionKeshav Santhanam∗Stanford UniversityOmar Khattab∗Stanford UniversityJon Saad-FalconGeorgia Institute of TechnologyChristopher PottsStanford UniversityMatei ZahariaStanford UniversityAbstractNeural information retrieval (IR) has greatlyadvanced search and other knowledge-intensive language tasks. While many neuralIR methods encode queries and documentsinto single-vector representations, lateinteraction models produce multi-vector repre-sentations at the granularity of each token anddecompose relevance modeling into scalabletoken-level computations. This decompositionhas been shown to make 

Now, we can feed to the Vespa instance (`app`), using the `feed_iterable` API, using the generator function above as input
with a custom `callback` function. Vespa also performs embedding inference during this step using the built-in Vespa [embedding](https://docs.vespa.ai/en/embedding.html#huggingface-embedder) functionality.

In [17]:
from vespa.io import VespaResponse

def callback(response:VespaResponse, id:str):
    if not response.is_successful():
        print(f"Document {id} failed to feed with status code {response.status_code}, url={response.url} response={response.json}")

app.feed_iterable(schema="pdf", iter=vespa_feed("jo-bergum"), namespace="personal", callback=callback)

Notice the `schema` and `namespace` arguments. PyVespa transforms the input operations to Vespa [document v1](https://docs.vespa.ai/en/document-v1-api-guide.html)
requests.

![Document id](https://blog.vespa.ai/assets/2023-12-08-turbocharge-rag-with-langchain-and-vespa-streaming-mode/docid.png)

### Querying data

Now, we can also query our data. With [streaming mode](https://docs.vespa.ai/en/reference/query-api-reference.html#streaming),
we must pass the `groupname` parameter, or the request will fail with an error.

The query request uses the Vespa Query API  and the `Vespa.query()` function
supports passing any of the Vespa query API parameters.

Read more about querying Vespa in:

- [Vespa Query API](https://docs.vespa.ai/en/query-api.html)
- [Vespa Query API reference](https://docs.vespa.ai/en/reference/query-api-reference.html)
- [Vespa Query Language API (YQL)](https://docs.vespa.ai/en/query-language.html)

Sample query request for `why is colbert effective?` for the user `bergum@vespa.ai`:

In [18]:
from vespa.io import VespaQueryResponse
import json

response:VespaQueryResponse = app.query(
    yql="select id,title,page,chunkno,chunk from pdf where userQuery() or ({targetHits:10}nearestNeighbor(embedding,q))",
    groupname="jo-bergum",
    ranking="colbert",
    query="why is colbert effective?",
    body={
        "presentation.format.tensors": "short-value",
        "input.query(q)": "embed(e5, \"why is colbert effective?\")",
        "input.query(qt)": "embed(colbert, \"why is colbert effective?\")",
    }
)
assert(response.is_successful())
print(json.dumps(response.hits[0], indent=2))

{
  "id": "id:personal:pdf:g=jo-bergum:6751328ffadff3bdf51bc963d0287ef533844b91",
  "relevance": 113.11050415039062,
  "source": "pdfs_content.pdf",
  "fields": {
    "matchfeatures": {
      "cos_sim": 0.6499400742312476,
      "max_sim": 113.11050415039062,
      "max_sim_per_context": {
        "0": 100.99246215820312,
        "1": 113.11050415039062
      }
    },
    "id": "6751328ffadff3bdf51bc963d0287ef533844b91",
    "title": "ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT",
    "page": 6
  }
}


Notice the `matchfeatures` that returns the configured match-features from the rank-profile, including all the chunk similarities.

## LangChain Retriever

We use the [LangChain Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/) interface so that
we can connect our Vespa app with the flexibility and power of the [LangChain](https://python.langchain.com/docs/get_started/introduction) LLM framework.

>A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

The retriever interface fits perfectly with Vespa, as Vespa can support a wide range of features and ways to retrieve and
rank content. The following implements a custom retriever `VespaStreamingHybridRetriever` that takes the following arguments:

- `app:Vespa` The Vespa application we retrieve from. This could be a Vespa Cloud instance or a local instance, for example running on a laptop.
-  `user:str` The user that that we want to retrieve for, this argument maps to the [Vespa streaming mode groupname parameter](https://docs.vespa.ai/en/reference/query-api-reference.html#streaming.groupname)
- `chunks:int` The  target number of text chunks from the PDFs we want to retrieve for a given query

The core idea is to _retrieve_ pages using chunk similarity as the initial scoring function, then re-rank the top-K chunks using the ColBERT embedding. This re-ranking is handled by the second phase of the Vespa ranking expression defined above, and is transparent to the retriever code below.

In [36]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from typing import List


class VespaStreamingHybridRetriever(BaseRetriever):

    app: Vespa
    user:str
    pages: int = 5
    chunks_per_page: int = 3
    chunk_similarity_threshold: float = 0.8

    def _get_relevant_documents(self, query: str) -> List[Document]:
        response:VespaQueryResponse = self.app.query(
            yql="select id, url, title, page, authors, chunks from pdf where userQuery() or ({targetHits:20}nearestNeighbor(embedding,q))",
            groupname=self.user,
            ranking="colbert",
            query=query,
            hits = self.pages,
            body={
                "presentation.format.tensors": "short-value",
                "input.query(q)": f"embed(e5, \"query: {query} \")",
                "input.query(qt)": f"embed(colbert, \"{query}\")"
            }
        )
        if not response.is_successful():
            raise ValueError(f"Query failed with status code {response.status_code}, url={response.url} response={response.json}")
        return self._parse_response(response)


    def _parse_response(self, response: VespaQueryResponse) -> List[Document]:
        documents: List[Document] = []
        for hit in response.hits:
            fields = hit['fields']
            chunks_with_scores = self._get_chunk_similarities(fields)
            ## Best k chunks from each page
            best_chunks_on_page = " ### ".join(
                [chunk for chunk, score in
                    chunks_with_scores[0:self.chunks_per_page] if score > self.chunk_similarity_threshold])
            documents.append(
                Document(
                    id=fields['id'],
                    page_content=best_chunks_on_page,
                    title=fields['title'],
                    metadata={
                        "title": fields['title'],
                        "url": fields['url'],
                        "page": fields['page'],
                        "authors": fields['authors'],
                        "features": fields['matchfeatures']
                    }
                )
            )
        return documents

    def _get_chunk_similarities(self, hit_fields: dict) -> List[tuple]:
        match_features = hit_fields['matchfeatures']
        similarities = match_features['max_sim_per_context']
        chunk_scores = []
        for i in range(0,len(similarities)):
            chunk_scores.append(similarities.get(str(i), 0))
        chunks = hit_fields['chunks']
        chunks_with_scores = list(zip(chunks, chunk_scores))
        return sorted(chunks_with_scores, key=lambda x: x[1], reverse=True)

That's it! We can give our newborn retriever a spin for the user `jo-bergum` by

In [37]:
vespa_hybrid_retriever = VespaStreamingHybridRetriever(app=app, user="jo-bergum", chunks=3)

In [38]:
vespa_hybrid_retriever.get_relevant_documents("what is the maxsim operator in colbert?")

[('ture that precisely does so. As illustrated, every query embeddinginteracts with all document embeddings via a MaxSim operator,which computes maximum similarity (e.g., cosine similarity), andthe scalar outputs of these operators are summed across queryterms. /T_his paradigm allows ColBERT to exploit deep LM-basedrepresentations while shi/f_ting the cost of encoding documents of-/f_line and amortizing the cost of encoding the query once acrossall ranked documents. Additionally, it enables ColBERT to lever-age vector-similarity search indexes (e.g., [ 1,15]) to retrieve thetop-kresults directly from a large document collection, substan-tially improving recall over models that only re-rank the output ofterm-based retrieval.As Figure 1 illustrates, ColBERT can serve queries in tens orfew hundreds of milliseconds. For instance, when used for re-ranking as in “ColBERT (re-rank)”, it delivers over 170 ×speedup(and requires 14,000 ×fewer FLOPs) relative to existing BERT-based', 119.84721374

[Document(page_content='ture that precisely does so. As illustrated, every query embeddinginteracts with all document embeddings via a MaxSim operator,which computes maximum similarity (e.g., cosine similarity), andthe scalar outputs of these operators are summed across queryterms. /T_his paradigm allows ColBERT to exploit deep LM-basedrepresentations while shi/f_ting the cost of encoding documents of-/f_line and amortizing the cost of encoding the query once acrossall ranked documents. Additionally, it enables ColBERT to lever-age vector-similarity search indexes (e.g., [ 1,15]) to retrieve thetop-kresults directly from a large document collection, substan-tially improving recall over models that only re-rank the output ofterm-based retrieval.As Figure 1 illustrates, ColBERT can serve queries in tens orfew hundreds of milliseconds. For instance, when used for re-ranking as in “ColBERT (re-rank)”, it delivers over 170 ×speedup(and requires 14,000 ×fewer FLOPs) relative to existing BERT

## RAG


Finally, we can connect our custom retriever with the complete flexibility and power of the [LangChain] LLM framework.
The following uses [LangChain Expression Language, or LCEL](https://python.langchain.com/docs/expression_language/), a declarative way to compose chains.

We have several steps composed into a chain:

- The prompt template and LLM model, in this case using OpenAI
- The retriever that provides the retrieved context for the question
- The formatting of the retrieved context



In [29]:
vespa_hybrid_retriever = VespaStreamingHybridRetriever(app=app, user="jo-bergum", chunks=12)

In [32]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """
Answer the question based only on the following context.
Cite the page number and the url of the document you are citing.

{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

def format_prompt_context(docs) -> str:
    context = []
    for d in docs:
        context.append(f"{d.metadata['title']} by {d.metadata['authors']}\n")
        context.append(f"url: {d.metadata['url']}\n")
        context.append(f"page: {d.metadata['page']}\n")
        context.append(f"{d.page_content}\n\n")
    return "".join(context)


chain = (
    {"context": vespa_hybrid_retriever | format_prompt_context, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

### Interact with the chain

Now, we can start asking questions using the `chain` define above.

In [33]:
chain.invoke("what is colbert?")

"ColBERT is a novel ranking model that adapts deep language models, particularly BERT, for efficient retrieval by independently encoding queries and documents and employing a late interaction mechanism. It is designed to balance the quality and cost of neural information retrieval. \n\nsource: ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT by ['Omar Khattab', 'Matei Zaharia']\nurl: https://arxiv.org/pdf/2004.12832.pdf\npage: 1"

In [34]:
chain.invoke("what is the colbert maxsim operator")

"The ColBERT MaxSim operator computes maximum similarity (e.g., cosine similarity) between query embeddings and document embeddings, and the scalar outputs of these operators are summed across query terms. This paradigm allows ColBERT to exploit deep LM-based representations while shifting the cost of encoding documents offline and amortizing the cost of encoding the query once across all ranked documents.\n\nSource:\n- ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT by ['Omar Khattab', 'Matei Zaharia']\n- URL: https://arxiv.org/pdf/2004.12832.pdf\n- Page: 4"

In [35]:
chain.invoke("What is the difference between colbert and single vector representational models?")

"ColBERT and single-vector representational models differ in their approach to modeling relevance. Single-vector models encode each query and document into a single high-dimensional vector and model relevance as a dot product between these vectors. On the other hand, ColBERT uses late interaction, where queries and documents are encoded at a finer granularity into multi-vector representations, and relevance is estimated using rich yet scalable interactions between these sets of vectors. This allows ColBERT to decompose relevance modeling into token-level computations, reducing the burden on the encoder and providing added expressivity. \n\nSource:\n- ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction by ['Keshav Santhanam', 'Omar Khattab', 'Jon Saad-Falcon', 'Christopher Potts', 'Matei Zaharia']\n- Page: 1\n- URL: https://arxiv.org/pdf/2112.01488.pdf"


## Summary

Vespa’s streaming mode is a game-changer, enabling the creation of highly cost-effective RAG applications for naturally partitioned data. Now it is also possible to use ColBERT for re-ranking, without having to integrate any custom embedder or re-ranking code.


In this notebook, we delved into the hands-on application of [LangChain](https://python.langchain.com/docs/get_started/introduction),
leveraging document loaders and transformers. Finally, we showcased a custom LangChain retriever that connected
all the functionality of LangChain with Vespa.

For those interested in learning more about Vespa, join the [Vespa community on Slack](https://vespatalk.slack.com/) to exchange ideas,
seek assistance, or stay in the loop on the latest Vespa developments.


We can now delete the cloud instance:

In [39]:
vespa_cloud.delete()

Deactivated vespa-team.pdfs in dev.aws-us-east-1c
Deleted instance vespa-team.pdfs.default
